### 🔨 Hands-on II: implementing predator-prey model

Let's do some scientific programming in Julia!

- we'll implement the so-called predator-prey model
- it describes the dynamics of a biological system with two interacting species

- a predator 🦊 ...

- and a prey 🐰

### Lotka-Volterra model

The predator-prey system can be described as a system of ODEs, called a _Lotka-Volterra model_:

$$
\frac{\mathrm{d}}{\mathrm{d}t}🐰 = \alpha \cdot 🐰 - \beta \cdot 🐰 \cdot 🦊
$$

$$
\frac{\mathrm{d}}{\mathrm{d}t}🦊 = \delta \cdot 🐰 \cdot 🦊 - \gamma \cdot 🦊
$$

- $\alpha$ and $\gamma$ are prey's growth rate and predator's death rate
- $\beta$ and $\delta$ are interaction parameters between species

### Expected output

<center><img src="./figures/l1_predator-prey.png" alt="predator-prey" width="50%"/><center/>

### Tasks

1. Implement the right-hand side of Lotka-Volterra system inside the functions `dx_dt`, `dx_dt`, and `predator_prey`

2. Implement the time integration loop within a single cell
    1. Measure the execution time with the `@time` macro
    2. Experiment with the time step by changing `nt`, see how the solution changes with decreasing `Δt`

    > The implementation is very similar to the one for the Lorenz attractor from the introduction

3. Implement non-allocating version of the time integration in the function `integrate!`
    1. Split the line within the time loop into two, updating the population of each species independently
    2. Call `integrate!`, time the results with the `@time` macro, compare to the execution time in the global scope
    3. Replace `out[it-1, 1]` with `out[it, 1]` in the second line (update rule for `out[it, 2]`) to make the integration semi-implicit

Let's introduce physical parameters:

In [ ]:
using CairoMakie, BenchmarkTools

# parameters
α = 1.1
β = 0.4
γ = 0.4
δ = 0.1

# total time of integration
ttot = 100.0

Then numerics:

In [ ]:
nt = 1000 # increase the number of time steps to get more accurate results
Δt = ttot / nt

And initial conditions:

In [ ]:
x0, y0 = 10.0, 10.0

Allocate arrays to store both explicit and semi-implicit solutions

In [ ]:
out1 = zeros(nt + 1, 2)
out2 = zeros(nt + 1, 2)

Let's implement the Lotka-Volterra equations:

In [ ]:
dx_dt(x, y, α, β) = ...
dy_dt(x, y, γ, δ) = ...

predator_prey(x, y, α, β, γ, δ) = ..., ...

Now we can solve ODEs in global scope using explicit Euler integration:

In [ ]:
out1[1, :] .= x0, y0
@time for it in 2:nt+1
    df_dt = ...
    out1[it, :] .= ...
end

Visualise the results:

In [ ]:
fig = Figure()

ax1 = Axis(fig[1, 1]; xlabel="time", ylabel="population", title="explicit Euler")

t1 = range(0, ttot, nt + 1)

series!(ax1, t1, out1'; labels=["prey", "predator"]);
axislegend(ax1);

fig

Well done! 🚀

However, the results don't look quite like we expect, both predator and prey populations seem to grow over time!

👉 Try to increase the number of time steps to `nt = 10000` and verify that the solution becomes more accurate.

Now let's implement the `integrate!` function. Use a `for` loop to avoid memory allocations:

In [ ]:
function integrate!(out, x0, y0, α, β, γ, δ, Δt, nt)
    out[1, :] .= x0, y0
    for it in 2:nt+1
        out[it, 1] = ...
        out[it, 2] = ...
    end
    t = range(0, nt * Δt, nt + 1)
    return t, out
end

Solve the same ODEs using the `integrate!` function and time the results:

In [ ]:
# ...

Make sure that the results are the same:

In [ ]:
@assert maximum(abs.(out1 .- out2)) < eps()

There is a simple trick to make the solution much more accurate without increasing the resolution too much.

First, revert `nt` to 1000 and re-run the previous simulation.

👉 Next, make a small modification to make the time integration semi-implicit:

In [ ]:
function integrate_si!(out, x0, y0, α, β, γ, δ, Δt, nt)
    out[1, :] .= x0, y0
    for it in 2:nt+1
        out[it, 1] = ...
        # hint: use the freshly computed value of x from the current time step
        # and the value of y from the previous time step
        out[it, 2] = ...
    end
    t = range(0, nt * Δt, nt + 1)
    return t, out
end

Re-run the code and add another panel with the results:

In [ ]:
@time t2, out2 = integrate_si!(...)

ax2 = Axis(fig[1, 2]; xlabel="time", ylabel="population", title="semi-implicit Euler")
series!(...; labels=["prey", "predator"])
axislegend(ax2)

fig

Much better, huh? Other way to explore the solutions to ODEs is visualising phase portraits:

In [ ]:
ax3 = Axis(fig[2, 1]; xlabel="prey", ylabel="predator")
ax4 = Axis(fig[2, 2]; xlabel="prey", ylabel="predator")

lines!(...)
lines!(...)

fig

Finally, let's benchmark the `integrate_si!` function (hint: use `@benchmark` macro and prepend each argument with '$')

In [ ]:
...